In [ ]:
from keras.models import Sequential, load_model
from keras.layers import *
from keras import optimizers
from keras import utils
from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau
import keras

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math

import tensorflow as tf

# Importing matplotlib to plot images.
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

# Importing SK-learn to calculate precision and recall
import sklearn
from sklearn import metrics
from sklearn.model_selection import train_test_split, cross_val_score, LeaveOneGroupOut
from sklearn.utils import shuffle 

# Used for graph export
from tensorflow.python.framework import graph_util
from tensorflow.python.framework import graph_io
from keras import backend as K

import pickle as pkl
import h5py

from pathlib import Path
import os.path
import sys
import datetime
import time



In [ ]:
train_ids = [1,2,3, 7, 8, 9, 10,12,13,14,15,16]
test_ids = [4,5,6,11,17]

In [ ]:
dfAll = pd.read_pickle("PklData/df_lstm_norm30.pkl")

In [ ]:
dfAll.TaskID[dfAll.TaskID>=17] = dfAll.TaskID - 17
dfNew = dfAll[dfAll.TaskID.isin([0, 2, 4,6,8,9,10,11,13,15])].copy()
dfNew.TaskID = dfNew.TaskID.replace([0, 2, 4,6,8,9,10,11,13,15], [0,1,2,3,4,5,6,7,8,9])

In [ ]:
dfAll = dfNew

In [ ]:
df_train = dfAll[dfAll.userID.isin(train_ids)]
df_test = dfAll[dfAll.userID.isin(test_ids)]

df_train2 = df_train[['Blobs', 'TaskID']].copy()
df_test2 = df_test[['Blobs', 'TaskID']].copy()

In [ ]:
x_train = np.concatenate(df_train2.Blobs.values).reshape(-1,30,27,15,1)
x_test = np.concatenate(df_test2.Blobs.values).reshape(-1,30,27,15,1)

y_train = df_train2.TaskID.values
y_test = df_test2.TaskID.values



In [ ]:
# convert class vectors to binary class matrices (one-hot notation)
num_classes = len(dfAll.TaskID.unique())
y_train_one_hot = utils.to_categorical(df_train2.TaskID, num_classes)
y_test_one_hot = utils.to_categorical(df_test2.TaskID, num_classes)

In [ ]:
num_classes

In [ ]:
# If GPU is not available: 
# GPU_USE = '/cpu:0'
#config = tf.ConfigProto(device_count = {"GPU": 1})


# If GPU is available: 
config = tf.ConfigProto()
config.log_device_placement = True
config.allow_soft_placement = True
config.gpu_options.allocator_type = 'BFC'

# Limit the maximum memory used
config.gpu_options.per_process_gpu_memory_fraction = 0.2

# set session config
tf.keras.backend.set_session(tf.Session(config=config))

In [ ]:
with tf.Session() as sess:
    model = load_model("lstm_beta.h5")
    sess.run(tf.global_variables_initializer())
    y_test_pred = model.predict(x_test)
    y_test_pred = np.argmax(y_test_pred, axis=1)
    #model.predict(x_test)
    
    print ('\n Summary of the precision, recall, F1 score for each class:')
    print (sklearn.metrics.classification_report(y_test, y_test_pred))

    print ('\n Confusion matrix: ')
    print (sklearn.metrics.confusion_matrix(y_test, y_test_pred))

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
#model = load_model("lstm_beta.h5")

    val_prediction =  model.predict(x_test)
    val_prediction = np.argmax(val_prediction, axis=1)

    train_prediction=model.predict(x_train)
    train_prediction = np.argmax(train_prediction, axis=1)


In [ ]:
val_pred = []
for i in val_prediction:
    a = np.zeros(10)
    np.put(a, i, 1)
    val_pred.append(a)

In [ ]:
len(y_test_one_hot)

In [ ]:
len(val_pred)

In [ ]:
#test
delta = val_pred - y_test_one_hot
indices_test = []
for i in range(len(val_pred)):
    if np.count_nonzero(delta[i]) > 0:
        indices_test += [i]

In [ ]:
#train
delta = train_prediction - y_train_one_hot
indices = []
for i in range(len(train_prediction)):
    if np.count_nonzero(delta[i]) > 0:
        indices += [i]

In [ ]:
len(indices_test)

In [ ]:
dfY = df_test[df_test.index.isin(indices_test)]

In [ ]:
dfX = df_train[df_train.index.isin(indices)]

In [ ]:
dfZ = pd.concat([dfX, dfY])

In [ ]:
dfY.groupby(['TaskID']).count()

In [ ]:
dfAll.groupby(['TaskID']).count()

In [ ]:
val_pred[0].argmax()

In [ ]:

plt.clf()
point = 1000
gesture = np.zeros((27,15))
for i in range(30):
    gesture += x_test[indices_test[point]][i].reshape(27,15)
print(val_pred[indices_test[point]].argmax())
print(y_test[indices_test[point]])

plt.figure(figsize=(6, 6))
ax = plt.gca()
plt.imshow(gesture, cmap='gray', vmin=0, vmax=255)
    # Loop over data dimensions and create text annotations.
    
plt.show()

In [ ]:
len(indices_test)